In [1]:
%matplotlib widget
from helper import *

<div style="font-family: 'Helvetica Neue', Arial, sans-serif;padding: 10px; text-align: center;">


<div style="margin-top: 10px; display: flex;">
    <div style="max-width: 500px; margin-right: auto">
        <img src="./Resources/neura_logo_black.png" alt="Neura Robotics Logo" style="max-width: 100%; height: auto; object-fit: contain;">
    </div>
    <div style="max-width: 600px; margin-left: auto">
        <img src="./Resources/rwth_igmr_de_rgb.png" alt="Neura Robotics Logo" style="max-width: 100%; height: auto; object-fit: contain;">
    </div>
</div>   
<div style="font-family: 'Helvetica Neue', Arial, sans-serif; max-width: 1500px; margin-left: auto; margin-right: auto ">
<h2 style = "text-align: center">Exploiting Redundancies with Predictive Differential Kinematics in Dual-Quaternion-Space for Configuration-optimized and Singularity-robust Trajectory Tracking</h2>
</div>

<div style="margin-bottom: 40px;">
    <!-- Using the logo from the data folder and making it bigger -->
    <img src="./Resources/maira.png" alt="Neura Robotics Logo" style="max-width: 1800px; width: 100%;">
</div>

</div>

# Problem Statement
---
$ q = (\cos\left(\frac{\theta}{2}\right), \sin\left(\frac{\theta}{2}\right)\hat{\mathbf{n}}) $


# Dual Quaternions
---
Three-dimensional rigid transformations are usually represented by homogeneous transformation matrices $T \in \mathcal{SE}(3)$. The *Special Euclidean Group* $\mathcal{SE}(3)$ is given by the semi-direct product of rotation matrices and translation vectors:

$$
\mathcal{SE}(3) = \mathcal{SO}(3) \ltimes \mathbb{R}^{3}
$$

The unit *Dual Quaternion Space* is then given by the semi-direct product of $\mathcal{S}^3$ and $\mathbb{R}^{3}$: 

$$
\mathbb{H} = \mathcal{S}^3 \ltimes \mathbb{R}^{3}
$$

With the space of unit quaternions $\mathcal{S}^3$ and the translation vectors $\mathbb{R}^{3}$.

# Dual Quaternions
---
A unit dual quaternion $\underline{\xi} \in \mathbb{H}$ can be written as

$$
\underline{\xi} = \hat{r} + \epsilon \frac{1}{2}\bar{t} \otimes \hat{r}
$$

where:
- unit quaternion $\hat{r} \in \mathcal{S}^3$
- pure quaternion $\bar{t} \in \mathbb{R}^{3}$
- dual unit $\epsilon$, with $\epsilon^2 = 0$ and  $\epsilon \neq 0$.

In [8]:
r = Quaternion(0.7071, 0.7071, 0, 0)
t = Quaternion(0, 1, 2, 0)

dual_quaternion = DualQuaternion(r, 0.5*t*r)
print(dual_quaternion)

DualQuaternion(Real: Quaternion(0.707, 0.707, 0.000, 0.000), Dual: Quaternion(-0.354, 0.354, 0.707, -0.707))


# Dual Quaternions: Double Cover Property
---

Dual Quaternions inherit all properties from the quaternions. These properties include:
- The **Double Cover Property** (or *Antipodal Property*)
- The **Singularity Free Representation**


In [2]:
#Example: Antipodal Property
q_start = Quaternion(1,0,0,0)
q_target = Quaternion(.900, 0.150, 0.000, 0.15).normalize()
q_target_antipodal = -1.0*q_target

fig, ax = create_3d_plot(q_target)

s_slider = create_slider("s", 0, 0, 1)
x_axis1, y_axis1, z_axis1 = draw_frame(ax, [0,0,0], q_start.asRotationMatrix())
x_axis2, y_axis2, z_axis2 = draw_frame(ax, [0,0,0], q_start.asRotationMatrix())

def update_plot(change):
    global x_axis1, y_axis1, z_axis1
    global x_axis2, y_axis2, z_axis2
    
    x_axis1.remove()
    y_axis1.remove()
    z_axis1.remove()

    x_axis2.remove()
    y_axis2.remove()
    z_axis2.remove()
    
    q_interpolated1 = Quaternion.slerp(q_start, q_target, s_slider.value)    
    q_interpolated2 = Quaternion.slerp(q_start, q_target_antipodal, s_slider.value)  
    
    x_axis1, y_axis1, z_axis1 = draw_frame(ax, [0,0,0], q_interpolated1.asRotationMatrix())
    x_axis2, y_axis2, z_axis2 = draw_frame(ax, [0,0,0], q_interpolated2.asRotationMatrix())

    fig.canvas.draw()
    fig.canvas.flush_events()

    
s_slider.observe(update_plot, names = 'value')

widgets.AppLayout(
    center=fig.canvas,
    footer=widgets.VBox([s_slider]),
    pane_heights=[0, 3, 1]
)

AppLayout(children=(VBox(children=(FloatSlider(value=0.0, description='s', layout=Layout(width='98%'), max=1.0…

# Dual Quaternions: Screw Transformation
---
<div style="font-family: 'Helvetica Neue', Arial, sans-serif; display: flex; justify-content: space-between;">

<!-- Left column for text -->
<div style="width: 48%;">

The screw axis $\vec{s} \in \mathbb{R}^6$ is defined from a unit rotation axis $\tilde{r}$ and a position $\vec{p}$

$$ 
\vec{s} = (\tilde{r}, \vec{p} \times \tilde{r}) = (\tilde{r}, \vec{m})
$$

Here, $\vec{m} = \vec{p} \times \tilde{r}$ is called the moment of the line.

As exponent for the screw transformation, the screw axis is often represented by the respective *Pure Dual Quaternion* $\underline{\bar{s}} = (0, \tilde{r}) + \epsilon (0, \vec{m}).$
    
</div>

<!-- Right column for image -->
<div style="width: 48%;">
    <img src="./Resources/screw.png" alt="calibration scheme" style="max-width: 500px; width: 100%; display: block; margin-left: auto; margin-right: auto;margin-top: 100px;">
    <p style="text-align: center; font-style: italic; margin-top: 10px;">Screw Axis example.</p>
</div>

# Dual Quaternions: Screw Transformation
---
A unit dual quaternion $ \underline{\xi} $ can then be computed from screw parameters with exponential mapping:

$$
\underline{\xi} = e^{\frac{\underline{\theta}}{2}\bar{\underline{s}}} = \cos(\frac{\underline{\theta}}{2}) + \bar{\underline{s}} \sin(\frac{\underline{\theta}}{2})
$$

with dual angle $\underline{\theta} = \theta + \epsilon d$ and pure dual quaternion (dual vector) $\bar{\underline{s}} = (0,\tilde{r}) + \epsilon (0,\vec{m})$.

dual angle:

- $\theta$: Angle of rotation around screw axis
- $d$: translation along screw axis ($d = 0$ for revolute joints)


In [2]:
fig, ax = create_3d_plot()

quaternion_display = create_textbox("screw")

angle_slider = create_slider("theta", 0, -2*np.pi, 2*np.pi)
azimuth_slider = create_slider("azimuth", 0.5, -2*np.pi, 2*np.pi)
elevation_slider = create_slider("elevation",1.2, -np.pi, np.pi)

x_slider = create_slider("x", .2, -1, 1)
y_slider = create_slider("y", .3, -1, 1)
z_slider = create_slider("z", 0, -1, 1)

rotation_axis = create_quiver(ax, [0,0,0], [1,0,0], 1, 'grey', "rotation_axis")
x_axis2, y_axis2, z_axis2 = draw_frame(ax, [0.1,.2,.1], np.eye(3))
dq_frame = DualQuaternion.fromQuatPos(Quaternion(1,0,0,0), [0.1,.2,.1])

rot_axis = ax.plot([-2, 2], [0,0], [0,0], "--", linewidth = 1, c = "k")
moment = ax.plot([0,0], [0,0], [0,0], "--", linewidth = 1, c = "k")

# Update function for the sliders
def update_plot(change):

    global rotation_axis, x_axis2, y_axis2, z_axis2, rot_axis, moment
    rotation_axis.remove()
    x_axis2.remove()
    y_axis2.remove()
    z_axis2.remove()
    rot_axis[0].remove()
    moment[0].remove()
    
    direction = spherical_coordinates(azimuth_slider.value, elevation_slider.value)
    
    # set position vector
    pos = [x_slider.value, y_slider.value, z_slider.value]
    
    screw_axis = DualQuaternion.screwAxis(*direction, *pos)
    
    theta = angle_slider.value
    d = 0.2*angle_slider.value
   
    exponent = DualQuaternion(screw_axis.real*theta, screw_axis.dual*theta + screw_axis.real*d)
    dq = DualQuaternion.exp(0.5*exponent)*dq_frame
    
    # update displays
    quaternion_display.value = str(screw_axis)
    
    # update the drawn vectors and the arc
    rotation_axis = create_quiver(ax, pos, direction,1, 'grey', "rotation_axis")
    
    rot_axis = ax.plot([pos[0]-direction[0]*2, pos[0]+direction[0]*2], [pos[1]-direction[1]*2, pos[1]+direction[1]*2], [pos[2]-direction[2]*2, pos[2]+direction[2]*2], "--", linewidth = 1, c = "black")
    moment = ax.plot([0, pos[0]], [0, pos[1]], [0, pos[2]], "--", linewidth = 1, c = "k")
 
    x_axis2, y_axis2, z_axis2 = draw_frame(ax, dq.getPosition().flatten(), dq.real.asRotationMatrix()*0.5)
    
    fig.canvas.draw()
    fig.canvas.flush_events()


angle_slider.observe(update_plot, names = 'value')
azimuth_slider.observe(update_plot, names='value')
elevation_slider.observe(update_plot, names='value')
x_slider.observe(update_plot, names='value')
y_slider.observe(update_plot, names='value')
z_slider.observe(update_plot, names='value')

widgets.AppLayout(
    center=fig.canvas,
    footer=widgets.VBox([quaternion_display, angle_slider, azimuth_slider, elevation_slider, x_slider, y_slider, z_slider]),
    pane_heights=[0, 2, 1])

AppLayout(children=(VBox(children=(Text(value='', description='screw', layout=Layout(width='98%')), FloatSlide…

# State of the Art: Online Motion Control
---

<a id="table21"></a>

<table>
    <thead>
      <tr>
        <th style="width: 200px";>Feature</th>
        <th>analytic IK</th>
        <th>iterative IK</th>
        <th>IDK (classic)</th>
        <th>IDK (QP)</th>
        <th>MPC</th>
        <th>NMPC</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td>real-time applicable</td>
        <td>yes</td>
        <td>no</td>
        <td>yes</td>
        <td>yes</td>
        <td>no</td>
        <td>no</td>
      </tr>
      <tr>
        <td>predictive approach</td>
        <td>no</td>
        <td>no</td>
        <td>no</td>
        <td>no</td>
        <td>yes</td>
        <td>yes</td>
      </tr>
      <tr>
        <td>nullspace projection</td>
        <td>no</td>
        <td>yes</td>
        <td>yes</td>
        <td>yes</td>
        <td>yes</td>
        <td>yes</td>
      </tr>
      <tr>
        <td>extra constraints</td>
        <td>no</td>
        <td>no</td>
        <td>no</td>
        <td>yes</td>
        <td>yes</td>
        <td>yes</td>
      </tr>
      <tr>
        <td>locally smooth</td>
        <td>no</td>
        <td>no</td>
        <td>yes</td>
        <td>yes</td>
        <td>yes</td>
        <td>yes</td>
      </tr>
    </tbody>
  </table>
  
<p style="text-align: center;"><strong>Table 2.1</strong>: Comparison of common inverse kinematic methods.</p>

# State of the Art: Dual Quaternion Interpolation
---

<a id="table22"></a>

<table>
    <thead>
      <tr>
        <th style="width: 200px";>Feature</th>
        <th>DLB</th>
        <th>ScLERP</th>
        <th>SEP(LERP)</th>
        <th>Bezier</th>
        <th>SCB</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td>decoupled interpolation</td>
        <td>no</td>
        <td>no</td>
        <td>yes</td>
        <td>yes</td>
        <td>-</td>
      </tr>
      <tr>
        <td>multipoint interpolation</td>
        <td>no</td>
        <td>no</td>
        <td>no</td>
        <td>yes</td>
        <td>yes</td>
      </tr>
      <tr>
        <td>definable motion profiles</td>
        <td>no</td>
        <td>no</td>
        <td>yes</td>
        <td>no</td>
        <td>-</td>
      </tr>
      <tr>
        <td>constant velocity</td>
        <td>no</td>
        <td>yes</td>
        <td>yes</td>
        <td>yes</td>
        <td>yes</td>
      </tr>
      <tr>
        <td>$\mathcal{C}^2$ continuity</td>
        <td>no</td>
        <td>no</td>
        <td>no</td>
        <td>no</td>
        <td>yes</td>
      </tr>
      <tr>
        <td>translation interpolation</td>
        <td>yes</td>
        <td>yes</td>
        <td>yes</td>
        <td>yes</td>
        <td>no</td>
      </tr>
      <tr>
        <td>orientation interpolation</td>
        <td>yes</td>
        <td>yes</td>
        <td>yes</td>
        <td>yes</td>
        <td>yes</td>
      </tr>
    </tbody>
  </table>
  
  <p style="text-align: center"><strong>Table 2.2</strong>: Comparison of common dual quaternion interpolation methods</p>

# Dual Quaternion Jacobian
---
The relation of the twist and the dual quaternion derivative is recalled for this transformation into the dual quaternion space. 

$$
\begin{align*}
\underline{\dot{\xi}} &= \frac{1}{2} \underline{\bar{\Omega}}_s \otimes \underline{\xi}\\
&= \frac{1}{2} [\underline{\xi}]_R \cdot \underline{\bar{\Omega}}_{s[vec]}
\end{align*}
$$

Substitution of the twist $\vec{\Omega}_s = J_s \dot{\vec{\theta}}$ with the space Jacobian and reformulation yields gives the *Unit Dual Quaternion Jacobian* $J_\mathbb{H}$:

$$
\begin{align*}
\underline{\dot{\xi}} &= \left(\frac{1}{2} [\underline{\xi}]_R J_s \right) \dot{\vec{\theta}}\\
&= J_\mathbb{H}\dot{\vec{\theta}}
\end{align*}
$$

# Predictive Differential Kinematics: Quadratic Formulation
---


$$
\begin{align*}
\min_{x}& \; f(x) = \frac{1}{2} x^T P x + q^T x \\
\textrm{subject to}& \; l \leq Ax \leq u \\
\end{align*}
$$


# Predictive Differential Kinematics: Yoshikawa Manipulability Optimization
---

Yoshikawa manipulability Measure:

$$
m(\theta) = \sqrt{\det \left( J(\theta)J(\theta)^T \right)}
$$

Yoshikawa Manipulability gradient: 

$$
\dot{m}(\theta) = \frac{d m(\theta)}{d\vec{\theta}}\dot{\vec{\theta}} = J_m^T\dot{\vec{\theta}}
$$

with the formulation from Corke and Haviland:

$$
J_m(\theta) = \sqrt{ \det \left( J(\theta)J(\theta)^T \right)}
\begin{bmatrix}
\text{vec}(J(\theta)H_1(\theta)^T) & \text{vec}((J(\theta)J(\theta)^T)^{-1}) \\
\text{vec}(J(\theta)H_2(\theta)^T) & \text{vec}((J(\theta)J(\theta)^T)^{-1}) \\
\vdots & \vdots \\
\text{vec}(J(\theta)H_{DoF}(\theta)^T) & \text{vec}((J(\theta)J(\theta)^T)^{-1})
\end{bmatrix}
$$

# Predictive Differential Kinematics: Directional Manipulability Optimization
---

With the direction which is computed from the dual quaternionic body twist, 

$$
 dir = \left( o_x, o_y, o_z, t_x, t_y, t_z \right) = \underline{\bar{\Omega}}_{b[6vec]} = \left[ \underline{\xi}_{des}^*\otimes \underline{\dot{\xi}}_{des}\right]_{6vec}
$$

The directional Gradient can be computed via weighing and regularization of the classic formulation of the manipulability gradient:

$$
J_{dir}(\theta) = 
\begin{bmatrix}
\text{vec}(\color{red}{W_{dir}}\color{black}{} J(\theta)H_1(\theta)^T) & \text{vec}((J(\theta)J(\theta)^T  + \color{blue}{\lambda I}\color{black}{})^{-1}) \\
\text{vec}(\color{red}{W_{dir}}\color{black}{} J(\theta)H_2(\theta)^T) & \text{vec}((J(\theta)J(\theta)^T + \color{blue}{\lambda I}\color{black}{})^{-1}) \\
\vdots & \vdots \\
\text{vec}(\color{red}{W_{dir}}\color{black}{} J(\theta)H_{DoF}(\theta)^T) & \text{vec}((J(\theta)J(\theta)^T + \color{blue}{\lambda I}\color{black}{})^{-1}) 
\end{bmatrix}
$$

# Predictive Differential Kinematics: Prediction
---

$$
dir_{pred}(t) = \frac{ dir_0 + 2 \sum_{i=0}^{n} \left| \left[ \underline{\xi}_{des}^*(t + i \Delta t)\otimes \underline{\dot{\xi}}_{des}(t + i \Delta t)\right]_{6vec}\right|}{ \left\| dir_0 + 2 \sum_{i=0}^{n} \left[ \underline{\xi}_{des}^*(t + i \Delta t)\otimes \underline{\dot{\xi}}_{des}(t + i \Delta t)\right]_{6vec}\right\|} 
$$


# Predictive Differential Kinematics: proposed Method
---

$$
\begin{align*}
\min_{x} \; f(x) = &\frac{1}{2} 
\begin{bmatrix}
\dot{\theta}\\
\ddot{\theta}\\
s
\end{bmatrix}^T 
\begin{bmatrix}
W_{\dot{\theta}} & 0 & 0\\
0 & W_{\ddot{\theta}} & 0\\
0 & 0 & \lambda_s
\end{bmatrix}
\begin{bmatrix}
\dot{\theta}\\
\ddot{\theta}\\
s
\end{bmatrix} + 
\begin{bmatrix}
(W_p \theta + \lambda_l J_l(\theta) - \lambda_{dir}J_{dir}(\theta, dir))\\
W_v \dot{\theta} \\
-2\lambda_s
\end{bmatrix}^T 
\begin{bmatrix}
\dot{\theta}\\
\ddot{\theta}\\
s
\end{bmatrix}\\
\\
\textrm{subject to}& \; 
\begin{bmatrix}
0_{8 \times 1}\\
\dot{\theta}_{k-1}\\
\dot{\theta}_m\\
-inf\\
0
\end{bmatrix} 
\leq 
\begin{bmatrix}
J_\mathbb{H}(\theta) & 0_{8 \times DoF} &-[\dot{\underline{\xi}}_{des} + k_p\left(\underline{\xi}_{des} - fk(\theta)\right)]_{vec}\\
I_{DoF \times DoF} & -\Delta t I_{DoF \times DoF} & 0_{DoF \times 1}\\
I_{DoF \times DoF} & 0_{DoF \times DoF} & 0_{DoF \times 1}\\
0_{DoF \times DoF} & I_{DoF \times DoF} & 0_{DoF \times 1}\\
0_{1 \times DoF} & 0_{1 \times DoF} & 1\\
\end{bmatrix} 
\begin{bmatrix}
\dot{\theta}\\
\ddot{\theta}\\
s
\end{bmatrix} \leq 
\begin{bmatrix}
0_{8 \times 1}\\
\dot{\theta}_{k-1}\\
\dot{\theta}_M\\
inf\\
1
\end{bmatrix} 
\end{align*}
$$

# Predictive Differential Kinematics: Comparison to Yoshikawa manipulability
---


# Predictive Differential Kinematics
---


<div style="font-family: 'Helvetica Neue', Arial, sans-serif;">
<h1 style="border-bottom: 2px solid #ddd; padding-bottom: 10px;">Orientation Representation with Unit Quaternions: Axis Angle</h1>
    
<p>Given an unit-length axis $ \hat{\mathbf{n}} = [n_x, n_y, n_z] $ and an angle $ \theta $, the corresponding quaternion $ q =(w, \mathbf{v}) $ is given by:</p>
<p>$$ q = (\cos\left(\frac{\theta}{2}\right), \sin\left(\frac{\theta}{2}\right)\hat{\mathbf{n}}) $$</p>
</div>

<div style="font-family: 'Helvetica Neue', Arial, sans-serif;">
<h1 style="border-bottom: 2px solid #ddd; padding-bottom: 10px;">Orientation Representation with Unit Quaternions: Axis Angle</h1>
    
<p>Given an unit-length axis $ \hat{\mathbf{n}} = [n_x, n_y, n_z] $ and an angle $ \theta $, the corresponding quaternion $ q =(w, \mathbf{v}) $ is given by:</p>
<p>$$ q = (\cos\left(\frac{\theta}{2}\right), \sin\left(\frac{\theta}{2}\right)\hat{\mathbf{n}}) $$</p>
</div>

<div style="font-family: 'Helvetica Neue', Arial, sans-serif;">
<h1 style="border-bottom: 2px solid #ddd; padding-bottom: 10px;">Orientation Representation with Unit Quaternions: Axis Angle</h1>
    
<p>Given an unit-length axis $ \hat{\mathbf{n}} = [n_x, n_y, n_z] $ and an angle $ \theta $, the corresponding quaternion $ q =(w, \mathbf{v}) $ is given by:</p>
<p>$$ q = (\cos\left(\frac{\theta}{2}\right), \sin\left(\frac{\theta}{2}\right)\hat{\mathbf{n}}) $$</p>
</div>

<div style="font-family: 'Helvetica Neue', Arial, sans-serif;">
<h1 style="border-bottom: 2px solid #ddd; padding-bottom: 10px;">Orientation Representation with Unit Quaternions: Axis Angle</h1>
    
<p>Given an unit-length axis $ \hat{\mathbf{n}} = [n_x, n_y, n_z] $ and an angle $ \theta $, the corresponding quaternion $ q =(w, \mathbf{v}) $ is given by:</p>
<p>$$ q = (\cos\left(\frac{\theta}{2}\right), \sin\left(\frac{\theta}{2}\right)\hat{\mathbf{n}}) $$</p>
</div>

<div style="font-family: 'Helvetica Neue', Arial, sans-serif;">
<h1 style="border-bottom: 2px solid #ddd; padding-bottom: 10px;">Orientation Representation with Unit Quaternions: Axis Angle</h1>
    
<p>Given an unit-length axis $ \hat{\mathbf{n}} = [n_x, n_y, n_z] $ and an angle $ \theta $, the corresponding quaternion $ q =(w, \mathbf{v}) $ is given by:</p>
<p>$$ q = (\cos\left(\frac{\theta}{2}\right), \sin\left(\frac{\theta}{2}\right)\hat{\mathbf{n}}) $$</p>
</div>

<div style="font-family: 'Helvetica Neue', Arial, sans-serif;">
<h1 style="border-bottom: 2px solid #ddd; padding-bottom: 10px;">Orientation Representation with Unit Quaternions: Axis Angle</h1>
    
<p>Given an unit-length axis $ \hat{\mathbf{n}} = [n_x, n_y, n_z] $ and an angle $ \theta $, the corresponding quaternion $ q =(w, \mathbf{v}) $ is given by:</p>
<p>$$ q = (\cos\left(\frac{\theta}{2}\right), \sin\left(\frac{\theta}{2}\right)\hat{\mathbf{n}}) $$</p>
</div>